# Imports

In [141]:
import pandas as pd
import numpy as np
import ast
import nltk
import pickle


# Load Data

In [13]:
movie = pd.read_csv("tmdb_5000_movies.csv")
credit = pd.read_csv("tmdb_5000_credits.csv" ,header = 0, encoding='utf-8')

In [14]:
movie.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [15]:
credit.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


# Data Preprocessing

In [56]:
movie_df = movie.merge(credit, on = 'title')
print("movie dataset shape = ", movie.shape)
print("credit dataset shape = ", credit.shape)
print("movie and credit after joining dataset shape = ", movie_df.shape)

movie dataset shape =  (4803, 20)
credit dataset shape =  (4803, 4)
movie and credit after joining dataset shape =  (4809, 23)


In [57]:
movie_df = movie_df[['movie_id','title','overview','genres','keywords','cast','crew']]

In [58]:
movie_df.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [59]:
movie_df.dropna(inplace=True)

<ipython-input-59-6436970ac8a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df.dropna(inplace=True)


**Convert each dict value column and extract only the neccessary data what we need for movie recommendation**

In [60]:
def convert_genres(genres):
  extracted_category = []
  for i in ast.literal_eval(genres):
    extracted_category.append(i['name'])
  return extracted_category

def convert_cast(cast):
  extracted_category = []
  for i in ast.literal_eval(cast)[0:3]:
    extracted_category.append(i['name'])
  return extracted_category

def convert_crew(crew):
  extracted_category = []
  for i in ast.literal_eval(crew):
    if i['job']=='Director':
      extracted_category.append(i['name'])
  return extracted_category

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1


In [61]:
convert_genres('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [62]:
movie_df['genres'] = movie_df['genres'].apply(convert_genres)
movie_df['keywords'] = movie_df['keywords'].apply(convert_genres)

In [63]:
movie_df['cast'] = movie_df['cast'].apply(convert_cast)

In [64]:
movie_df['crew'] = movie_df['crew'].apply(convert_crew)

In [65]:
movie_df.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [66]:
# movie_df['overview'] = movie_df['overview'].apply(collapse)
movie_df['genres'] = movie_df['genres'].apply(collapse)
movie_df['keywords'] = movie_df['keywords'].apply(collapse)
movie_df['cast'] = movie_df['cast'].apply(collapse)
movie_df['crew'] = movie_df['crew'].apply(collapse)

In [67]:
movie_df.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [68]:
movie_df['overview'] = movie_df['overview'].apply(lambda x: x.split())

In [69]:
movie_df.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [70]:
movie_df['tags'] = movie_df['overview'] + movie_df['genres'] + movie_df['keywords'] + movie_df['cast'] + + movie_df['crew']
movie_df_filtered = movie_df.drop(columns=['overview','genres','keywords','cast','crew'])

In [71]:
movie_df_filtered.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [75]:
movie_df_filtered['tags'] = movie_df_filtered['tags'].apply(lambda x : " ".join(x))
movie_df_filtered.head(1)

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


In [81]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [84]:
def stem(text):
  stemed_sentence = []

  for i in text.split():
    stemed_sentence.append(ps.stem(i))
  return " ".join(stemed_sentence)


In [85]:
movie_df_filtered['tags'] = movie_df_filtered['tags'].apply(stem)
movie_df_filtered.head(1)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."


# CountVectorization

In [87]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [88]:
vector_tags = cv.fit_transform(movie_df_filtered['tags']).toarray()

In [89]:
vector_tags.shape

(4806, 5000)

# Cosine Similarity

In [90]:
from sklearn.metrics.pairwise import cosine_similarity


In [118]:
cos_similarity = cosine_similarity(vector_tags)

In [119]:
cos_similarity.shape

(4806, 4806)

In [133]:
cos_similarity_df = pd.DataFrame(cos_similarity)
# cos_similarity_df = cos_similarity_df.add_suffix('_index')
cos_similarity_df.reset_index(inplace=True)

# Recommendations

In [138]:
def recommendation(title):
  pick_index = movie_df_filtered[movie_df_filtered['title']=='Avatar'].index[0]
  top_recommended_df = pd.DataFrame(cos_similarity_df[cos_similarity_df['index']==pick_index].T[2:].reset_index()).rename(columns={0:'similarity'}).sort_values(by='similarity', ascending=False)[0:6]
  for idx in top_recommended_df['index'].values:
    print(movie_df_filtered.iloc[idx]['title'])

In [140]:
recommendation('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.
Battle: Los Angeles


In [115]:

def recommend(movie):
    index = movie_df_filtered[movie_df_filtered['title'] == movie].index[0]
    distances = sorted(list(enumerate(cos_similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        # print(movie_df_filtered.iloc[i[0]].title)
        print(i[0])

In [116]:
recommend('Avatar')

1216
2409
3730
507
539


# Saving the Processed recommended df and the similarity df

In [142]:
pickle.dump(movie_df_filtered,open('movie_list.pkl','wb'))
pickle.dump(cos_similarity_df,open('similarity.pkl','wb'))